<a href="https://colab.research.google.com/github/risa1796/Hotel-Recommendation-System/blob/main/Hotels_near_me_recommendation_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hotel Recommendation 

- Hotels near my location

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/딥러닝기초/ HotelGo

/content/drive/MyDrive/딥러닝기초/ HotelGo


## Data Upload

In [3]:
import pandas as pd

In [4]:
hotel_details = pd.read_csv('Hotel_details.csv')
hotel_details.head()

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR


In [5]:
hotel_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108048 entries, 0 to 108047
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            108048 non-null  int64  
 1   hotelid       108048 non-null  int64  
 2   hotelname     108048 non-null  object 
 3   address       102955 non-null  object 
 4   city          108048 non-null  object 
 5   country       108048 non-null  object 
 6   zipcode       83486 non-null   float64
 7   propertytype  108048 non-null  object 
 8   starrating    108048 non-null  int64  
 9   latitude      108048 non-null  float64
 10  longitude     108048 non-null  float64
 11  Source        108048 non-null  int64  
 12  url           107937 non-null  object 
 13  curr          108048 non-null  object 
dtypes: float64(3), int64(4), object(7)
memory usage: 11.5+ MB


In [6]:
# View columns by their data types
for dtype, cols in hotel_details.groupby(by=hotel_details.dtypes, axis=1):
  print(dtype)
  for col in list(cols):
    print(f'unique value of column "{col}" : ', hotel_details[col].nunique())
  print('---')

int64
unique value of column "id" :  108048
unique value of column "hotelid" :  106193
unique value of column "starrating" :  3
unique value of column "Source" :  2
---
float64
unique value of column "zipcode" :  17245
unique value of column "latitude" :  106881
unique value of column "longitude" :  107088
---
object
unique value of column "hotelname" :  96203
unique value of column "address" :  96978
unique value of column "city" :  25693
unique value of column "country" :  29
unique value of column "propertytype" :  8
unique value of column "url" :  107936
unique value of column "curr" :  8
---


In [7]:
duplicate_df = pd.DataFrame(hotel_details.hotelid.value_counts(ascending=False)).reset_index()
duplicate_df[duplicate_df['hotelid']>1]

,index,hotelid
0,172102,2
1,66032,2
2,191655,2
3,60055,2
4,366853,2
...,...,...
1850,231785,2
1851,71171,2
1852,89194,2
1853,325864,2


In [8]:
hotel_details[hotel_details['hotelid']==172102]

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
104214,150620,172102,Best Western Hotel Rocca,Via Sferracavalli 105,Cassino,Italy,3043.0,Hotels,4,41.507394,13.833987,2,http://www.booking.com/hotel/it/rocca.en-gb.html,EUR
104215,150621,172102,The Coachman Hotel,4 Parkfoot Street,Kilsyth,United Kingdom,NaN,Hotels,3,55.978600,-4.058210,5,https://www.agoda.com/en-gb/the-coachman-hotel...,GBP


HotelId is is not unique

In [9]:
hotel_details.id.duplicated().sum()

0

Id is unique, indeed. We will delete hotelid column.

In [10]:
hotel_details.drop('hotelid',axis=1, inplace=True)

In [11]:
hotel_details.starrating.value_counts()

3    56419
4    35892
2    15737
Name: starrating, dtype: int64

In [12]:
hotel_details.Source.value_counts()

2    68882
5    39166
Name: Source, dtype: int64

## Hotels near my location 

In [13]:
#! pip install geocoder

In [14]:
# Reference: https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [15]:
def find_nearest(lat, long):
    distances = hotel_details.apply(
        lambda row: dist(lat, long, row['latitude'], row['longitude']), 
        axis=1)
    
    df = pd.DataFrame(distances,columns=['km_diff'])
    top10 = df.sort_values(by='km_diff')[:11]
    nearest_hotel = distances.idxmin()
    top10_index = top10.index.tolist()

    #display(hotel_details.loc[top10_index, 'hotelname'])
    #print('Nearest Hotel : ', hotel_details.loc[distances.idxmin(), 'hotelname'])
    return top10_index

In [16]:
#import geocoder
#g = geocoder.ip('me')
#my_position = g.latlng

my_position = [52.523430,13.411440] #example location

In [17]:
top10_index = find_nearest(my_position[0],my_position[1])

In [18]:
#! pip install folium

In [19]:
import folium

map = folium.Map(location=[my_position[0], my_position[1]], zoom_start=15)
marker = folium.Marker([my_position[0],my_position[1]], icon=folium.Icon(icon='home',color='blue')).add_to(map)

tooltip = "Click me!"
for idx, i in enumerate(top10_index):
    name = hotel_details.loc[i, 'hotelname']
    lat = hotel_details.loc[i, 'latitude']
    lon = hotel_details.loc[i, 'longitude']
    url = hotel_details.loc[i,'url']

    full_url = "<a href=" + url + " target=_blank>" + name + " </a>"
    if idx == 0: # Nearest Hotel 
      marker = folium.Marker([lat,lon], popup=full_url,icon=folium.Icon(icon='fa-regular fa-hotel',prefix='fa',color='red')).add_to(map)
    else:
      marker = folium.Marker([lat,lon],tooltip=tooltip ,popup=full_url,icon=folium.Icon(icon='fa-regular fa-hotel',prefix='fa',color='orange')).add_to(map)

map

--------